# API - Weather Data

## 1. Retrieve city data from MySQL database

In [ ]:
# import libraries
import pandas as pd
import requests
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime

In [ ]:
# Connection setup MySQL
schema = "city_infos"
host = "127.0.0.1"
user = "root"
password = os.getenv("MYSQL_PASSWORD") # password = "YOUR_PASSWORD"
port = 3306

# Create connection string for MySQL
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# Connection for API
key = os.getenv("OPENWEATHERMAP_API_KEY") # key = "YOUR_OPENWEATHERMAP_API_KEY"

In [ ]:
# retrieve city data
cities_df = pd.read_sql('cities', con=connection_string)
cities_df

## 2. API call for weather data

### Explore response for one city

In [ ]:
# retrieve weather data for Berlin from OpenWeather API (3h forecast for the next 5 days)

# pick latitude, longitude for Berlin example
lat = cities_df.loc[0,'latitude']
lon = cities_df.loc[0,'longitude']

URL = f"https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={key}&units=metric"
berlin_response = requests.get(URL)

In [ ]:
# Transform the response for exploration
berlin_response_json = berlin_response.json()
#berlin_response_json

In [ ]:
# check which keys are available
berlin_response_json.keys()

In [ ]:
# exploring how to extract the weather data
berlin_response_json['list'][0]['main']

In [ ]:
# for each entry there is a date and the corresponding weather data
for entry in berlin_response_json['list']:
    print(entry['dt_txt'])  # get the date & time
    print(entry['weather'][0]['description'])
    print(entry['main']['temp'])
    print(entry['wind']['speed'])
    print(entry['pop'])

In [ ]:
# create data frame for the extracted weather data for Berlin (3h forecast for the next 5 days)
berlin_weather_data = []

for entry in berlin_response_json.get("list", []): # Use .get() to avoid errors 
    weather_entry = {
        "city_id": 1,    # for Berlin Example
        "time": entry.get("dt_txt", None),
        "weather": entry.get("weather", [{}])[0].get("description", None),
        "temperature_c": entry.get("main", {}).get("temp", None),
        "wind_speed_m_s": entry.get("wind", {}).get("speed", None),
        "rain_probability": entry.get("pop", 0),
        "retrieval_time": datetime.today().strftime("%Y-%m-%d %H:%M:%S") # add retrieval time
    }
    berlin_weather_data.append(weather_entry)

berlin_weather_df = pd.DataFrame(berlin_weather_data)
berlin_weather_df.head()

### Create function for multiple cities

In [ ]:
cities_df

In [ ]:
# create function to collect weather data for cities_df

# import libraries
import pandas as pd
import requests
from sqlalchemy import create_engine, text
import os
from dotenv import load_dotenv
load_dotenv()
from datetime import datetime

# Connection setup MySQL
schema = "city_infos"
host = "127.0.0.1"
user = "root"
password = os.getenv("MYSQL_PASSWORD") # password = "YOUR_PASSWORD"
port = 3306

# Create connection string for MySQL
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# Connection for API
key = os.getenv("OPENWEATHERMAP_API_KEY") # key = "YOUR_OPENWEATHERMAP_API_KEY"

def get_weather (cities):
    
    weather_data = []   # retrieve data for all cities

    for i, row in cities.iterrows():
        URL = f"https://api.openweathermap.org/data/2.5/forecast?lat={row['latitude']}&lon={row['longitude']}&appid={key}&units=metric"
        response = requests.get(URL)
        if response.status_code == 200:     # Only extract data if call was successful
            response_json = response.json()

            for entry in response_json.get("list", []):
                weather_entry = {
                    "city_id": row['city_id'],
                    "time": entry.get("dt_txt", None),
                    "weather": entry.get("weather", [{}])[0].get("description", None),
                    "temperature_c": float(entry.get("main", {}).get("temp", None)),
                    "wind_speed_m_s": float(entry.get("wind", {}).get("speed", None)),
                    "rain_probability": float(entry.get("pop", 0)),
                    "retrieval_time": datetime.today().strftime("%Y-%m-%d %H:%M:%S")
                }
                weather_data.append(weather_entry) 
            weather_df = pd.DataFrame(weather_data)
            weather_df['time'] = pd.to_datetime(weather_df['time'])     # convert to datetime
            weather_df['retrieval_time'] = pd.to_datetime(weather_df['retrieval_time']) # convert to datetime

        else:
            print(f"Error {response.status_code} at {row['id']}") # error handling

    return(weather_df)

In [ ]:
# get cities_df from MySQL database
cities_df = pd.read_sql('cities', con=connection_string)

# call get_weather function
weather_df = get_weather(cities_df)

In [ ]:
weather_df

In [ ]:
weather_df.info()

## 3. Send weather data to MySQL database

In [ ]:
weather_df.to_sql('weather',
                 if_exists='append',
                 index=False,
                 con=connection_string)